In [1]:
import tensorflow as tf

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train/255, x_test/255

170500096/170498071 [==============================] - 6s 0us/step


In [3]:
print(x_train.shape)
print(y_train.shape)

(50000, 32, 32, 3)
(50000, 1)


In [4]:
y_train[:10]

array([[6],
       [9],
       [9],
       [4],
       [1],
       [1],
       [2],
       [7],
       [8],
       [3]], dtype=uint8)

In [5]:
from tensorflow import  keras
from tensorflow.keras import layers
from keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, Add, Activation
from keras.models import Model, Sequential

# **Residual Connection**

In [6]:
input_shape = (32,32,3)

model = Sequential()
model.add(Conv2D(128, 3, padding="same", activation='relu', input_shape=input_shape))
model.add(Conv2D(128, 3, padding="same", activation='relu'))
model.add(Conv2D(128, 3, padding="same", activation='relu'))
model.add(Conv2D(128, 3, padding="same", activation='relu'))
model.add(Conv2D(128, 3, padding="same", activation='relu'))
model.add(Conv2D(128, 3, padding="same", activation='relu'))
model.add(Conv2D(128, 3, padding="same", activation='relu'))
model.add(Conv2D(128, 3, padding="same", activation='relu'))
model.add(MaxPooling2D(strides=(2,2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 128)       3584      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 128)       147584    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 128)       147584    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 128)       147584    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 128)       147584    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 32, 128)       147584    
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 128)       1

In [7]:
model.compile(optimizer=tf.keras.optimizers.Adam(), 
            loss='sparse_categorical_crossentropy', 
            metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

model.fit(x_train, y_train, epochs=7)
model.evaluate(x_test, y_test, verbose=2)

Epoch 1/7
1563/1563 [==============================] - 133s 66ms/step - loss: 2.3024 - sparse_categorical_accuracy: 0.1004
Epoch 2/7
1563/1563 [==============================] - 102s 65ms/step - loss: 2.3027 - sparse_categorical_accuracy: 0.0986
Epoch 3/7
1563/1563 [==============================] - 102s 65ms/step - loss: 2.3027 - sparse_categorical_accuracy: 0.0992
Epoch 4/7
1563/1563 [==============================] - 102s 65ms/step - loss: 2.3028 - sparse_categorical_accuracy: 0.0978
Epoch 5/7
1563/1563 [==============================] - 102s 65ms/step - loss: 2.3026 - sparse_categorical_accuracy: 0.0987
Epoch 6/7
1563/1563 [==============================] - 103s 66ms/step - loss: 2.3028 - sparse_categorical_accuracy: 0.0983
Epoch 7/7
1563/1563 [==============================] - 103s 66ms/step - loss: 2.3028 - sparse_categorical_accuracy: 0.0995
313/313 - 5s - loss: 2.3026 - sparse_categorical_accuracy: 0.1000


[2.3026278018951416, 0.10000000149011612]

## **Shortcut Connection**

In [8]:
inputs = keras.Input(shape=(32, 32, 3))
x = inputs

x = Conv2D(128, 3, padding='same')(x)
x = Activation('relu')(x)
x = Conv2D(128, 3, padding='same')(x)
x = Activation('relu')(x)

x_skip = x
x = Conv2D(128, 3, padding='same')(x)
x = Activation('relu')(x)
x = Conv2D(128, 3, padding='same')(x)
x = Activation('relu')(x)
x_skip = Conv2D(128, 3, padding='same')(x_skip)
x = Add()([x, x_skip])
x = Activation('relu')(x)

x_skip = x
x = Conv2D(128, 3, padding='same')(x)
x = Activation('relu')(x)
x = Conv2D(128, 3, padding='same')(x)
x = Activation('relu')(x)
x_skip = Conv2D(128, 3, padding='same')(x_skip)
x = Add()([x, x_skip])
x = Activation('relu')(x)

x = layers.MaxPooling2D(2)(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(10, activation='softmax')(x)
outputs = x

model = keras.Model(inputs, outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 32, 32, 128)  3584        input_1[0][0]                    
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 128)  0           conv2d_8[0][0]                   
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 32, 32, 128)  147584      activation[0][0]                 
______________________________________________________________________________________________

In [9]:
model.compile(optimizer=tf.keras.optimizers.Adam(), 
            loss='sparse_categorical_crossentropy', 
            metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

model.fit(x_train, y_train, epochs=7)
model.evaluate(x_test, y_test, verbose=2)

Epoch 1/7
1563/1563 [==============================] - 108s 68ms/step - loss: 1.7475 - sparse_categorical_accuracy: 0.3546
Epoch 2/7
1563/1563 [==============================] - 107s 68ms/step - loss: 1.0948 - sparse_categorical_accuracy: 0.6100
Epoch 3/7
1563/1563 [==============================] - 107s 68ms/step - loss: 0.8183 - sparse_categorical_accuracy: 0.7111
Epoch 4/7
1563/1563 [==============================] - 107s 68ms/step - loss: 0.6369 - sparse_categorical_accuracy: 0.7758
Epoch 5/7
1563/1563 [==============================] - 107s 68ms/step - loss: 0.4731 - sparse_categorical_accuracy: 0.8351
Epoch 6/7
1563/1563 [==============================] - 107s 68ms/step - loss: 0.3470 - sparse_categorical_accuracy: 0.8773
Epoch 7/7
1563/1563 [==============================] - 107s 68ms/step - loss: 0.2489 - sparse_categorical_accuracy: 0.9141
313/313 - 6s - loss: 1.2745 - sparse_categorical_accuracy: 0.6777


[1.2745110988616943, 0.6776999831199646]

## **MaxPooling**

In [10]:
inputs = keras.Input(shape=(32, 32, 3))
x = inputs

x = Conv2D(128, 3, padding='same')(x)
x = Activation('relu')(x)
x = Conv2D(128, 3, padding='same')(x)
x = Activation('relu')(x)

x = layers.MaxPooling2D(2)(x)

x_skip = x
x = Conv2D(128, 3, padding='same')(x)
x = Activation('relu')(x)
x = Conv2D(128, 3, padding='same')(x)
x = Activation('relu')(x)
x_skip = Conv2D(128, 3, padding='same')(x_skip)
x = Add()([x, x_skip])
x = Activation('relu')(x)

x = layers.MaxPooling2D(2)(x)

x_skip = x
x = Conv2D(128, 3, padding='same')(x)
x = Activation('relu')(x)
x = Conv2D(128, 3, padding='same')(x)
x = Activation('relu')(x)
x_skip = Conv2D(128, 3, padding='same')(x_skip)
x = Add()([x, x_skip])
x = Activation('relu')(x)

x = layers.MaxPooling2D(2)(x)

x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(10, activation='softmax')(x)
outputs = x

model = keras.Model(inputs, outputs)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_16 (Conv2D)              (None, 32, 32, 128)  3584        input_2[0][0]                    
__________________________________________________________________________________________________
activation_8 (Activation)       (None, 32, 32, 128)  0           conv2d_16[0][0]                  
__________________________________________________________________________________________________
conv2d_17 (Conv2D)              (None, 32, 32, 128)  147584      activation_8[0][0]               
____________________________________________________________________________________________

In [11]:
model.compile(optimizer=tf.keras.optimizers.Adam(), 
            loss='sparse_categorical_crossentropy', 
            metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

model.fit(x_train, y_train, epochs=7)
model.evaluate(x_test, y_test, verbose=2)

Epoch 1/7
1563/1563 [==============================] - 45s 28ms/step - loss: 1.7183 - sparse_categorical_accuracy: 0.3628
Epoch 2/7
1563/1563 [==============================] - 44s 28ms/step - loss: 0.9431 - sparse_categorical_accuracy: 0.6668
Epoch 3/7
1563/1563 [==============================] - 44s 28ms/step - loss: 0.7276 - sparse_categorical_accuracy: 0.7425
Epoch 4/7
1563/1563 [==============================] - 43s 28ms/step - loss: 0.5841 - sparse_categorical_accuracy: 0.7966
Epoch 5/7
1563/1563 [==============================] - 43s 28ms/step - loss: 0.4745 - sparse_categorical_accuracy: 0.8340
Epoch 6/7
1563/1563 [==============================] - 43s 28ms/step - loss: 0.3930 - sparse_categorical_accuracy: 0.8649
Epoch 7/7
1563/1563 [==============================] - 43s 28ms/step - loss: 0.3186 - sparse_categorical_accuracy: 0.8889
313/313 - 3s - loss: 0.8329 - sparse_categorical_accuracy: 0.7500


[0.8329219818115234, 0.75]

## **MaxPooling & DropOut**

In [12]:
inputs = keras.Input(shape=(32, 32, 3))
x = inputs

x = Conv2D(128, 3, padding='same')(x)
x = Activation('relu')(x)
x = Conv2D(128, 3, padding='same')(x)
x = Activation('relu')(x)

x = layers.MaxPooling2D(2)(x)

x_skip = x
x = Conv2D(128, 3, padding='same')(x)
x = Activation('relu')(x)
x = Conv2D(128, 3, padding='same')(x)
x = Activation('relu')(x)
x_skip = Conv2D(128, 3, padding='same')(x_skip)
x = Add()([x, x_skip])
x = Activation('relu')(x)

x = layers.MaxPooling2D(2)(x)

x_skip = x
x = Conv2D(128, 3, padding='same')(x)
x = Activation('relu')(x)
x = Conv2D(128, 3, padding='same')(x)
x = Activation('relu')(x)
x_skip = Conv2D(128, 3, padding='same')(x_skip)
x = Add()([x, x_skip])
x = Activation('relu')(x)

x = layers.MaxPooling2D(2)(x)

x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(10, activation='softmax')(x)
outputs = x

model = keras.Model(inputs, outputs)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_24 (Conv2D)              (None, 32, 32, 128)  3584        input_3[0][0]                    
__________________________________________________________________________________________________
activation_16 (Activation)      (None, 32, 32, 128)  0           conv2d_24[0][0]                  
__________________________________________________________________________________________________
conv2d_25 (Conv2D)              (None, 32, 32, 128)  147584      activation_16[0][0]              
____________________________________________________________________________________________

In [14]:
model.compile(optimizer=tf.keras.optimizers.Adam(), 
            loss='sparse_categorical_crossentropy', 
            metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

model.fit(x_train, y_train, epochs=7)
model.evaluate(x_test, y_test, verbose=2)

Epoch 1/7
1563/1563 [==============================] - 45s 28ms/step - loss: 0.6190 - sparse_categorical_accuracy: 0.7928
Epoch 2/7
1563/1563 [==============================] - 44s 28ms/step - loss: 0.5614 - sparse_categorical_accuracy: 0.8131
Epoch 3/7
1563/1563 [==============================] - 44s 28ms/step - loss: 0.5229 - sparse_categorical_accuracy: 0.8236
Epoch 4/7
1563/1563 [==============================] - 44s 28ms/step - loss: 0.4843 - sparse_categorical_accuracy: 0.8359
Epoch 5/7
1563/1563 [==============================] - 44s 28ms/step - loss: 0.4579 - sparse_categorical_accuracy: 0.8461
Epoch 6/7
1563/1563 [==============================] - 44s 28ms/step - loss: 0.4403 - sparse_categorical_accuracy: 0.8524
Epoch 7/7
1563/1563 [==============================] - 44s 28ms/step - loss: 0.4259 - sparse_categorical_accuracy: 0.8565
313/313 - 3s - loss: 0.7871 - sparse_categorical_accuracy: 0.7480


[0.7870946526527405, 0.7480000257492065]